# IMPORTY

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
from modele_db import *
lista_bryl = dict()
with engine.begin() as conn:
      lb = conn.execute(text(f"SELECT * from lista_bryl_pianki"))
      tab = pd.read_sql(text("SELECT * from baza_PIANKI"), conn)
for i in lb:
  # print(i[0], i[1].split("_"))
  lista_bryl[i[0]] = [x for x in i[1].split("_")]

#PIANKI

In [3]:
from modele_pianek import *

# ANALIZA PIANEK

In [ ]:
# nr_tygodnia = 2409
# zapis_analizy =analiza[["KOD","MAX","SALDO","SUMA_ZLEC","ZAMOWIONE", "CZESIOWO_DOSTARCZONE", "CZEKA_NA_SPAKOWANIE",]]
# zapis_analizy["TYDZIEN"] = nr_tygodnia
# zapis_analizy[["MAX","SALDO","SUMA_ZLEC","ZAMOWIONE", "CZESIOWO_DOSTARCZONE", "CZEKA_NA_SPAKOWANIE"]] = zapis_analizy[["MAX","SALDO","SUMA_ZLEC","ZAMOWIONE", "CZESIOWO_DOSTARCZONE", "CZEKA_NA_SPAKOWANIE"]].astype("int16")
# zapis_analizy

# # zapis_analizy_engine = create_engine("sqlite:///drive/MyDrive/ARCHIWOM_ANALIZ.db", echo=False)
# # zapis_analizy.to_sql("ZAPIS_ANALIZY_PIANKI", zapis_analizy_engine, if_exists="append")

In [15]:
nr_pw = "24/14"
pw = pd.read_excel("Z:/450. PLANISTA - ZAOPATRZENIE/ZAM_PIANKI.xlsx", sheet_name="PW")
pw= pw[pw.PW.str.contains(nr_pw)].merge(komplety_pianek[["KOD", "obj"]], how="left", on="KOD")
pw["SPAKOWANE_M3"] = pw.ILOSC * pw.obj

print("OBIĘTOSC PACZEK (Z OWATAMI)")
sum_obj = 0
for p in pda:
  p_obj = (analiza[p]*analiza.obj).sum()
  sum_obj += p_obj
  print(f"{p}: {p_obj:.0f}m3")


print(f"WST: {(analiza.WST*analiza.obj).sum():.0f}m3")
print(f"Objetość wszystkich paczek: {sum_obj + (analiza.WST*analiza.obj).sum():.0f}m3")
print(f"PW {nr_pw} spakowano: {pw.SPAKOWANE_M3.sum():.0f}M3")
# pw

OBIĘTOSC PACZEK (Z OWATAMI)
05/01: 173m3
06/01: 184m3
07/01: 176m3
08/01: 176m3
09/01: 168m3
WST: 115m3
Objetość wszystkich paczek: 991m3
PW 24/14 spakowano: 265M3


In [ ]:
zam_pianki[zam_pianki.dostarczono == 2]

In [ ]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br#.to_excel("BRAKI.xlsx")
# br#.sort_values(by=["GRUPA", "OPIS"])

In [ ]:
Zagrozone(WOLNE="SALDO")

##ANALIZA RODZINY

In [ ]:
ar=[Analiza_Rodziny(AMALFI_P,2,1),
    Analiza_Rodziny(AVANT_P,1,1),
    Analiza_Rodziny(CALYPSO,3,2),
    Analiza_Rodziny(COCO_PV,4),
    Analiza_Rodziny(DIVA,2,10),
    Analiza_Rodziny(DUO,3),
    Analiza_Rodziny(ELIXIR,1,1),
    # Analiza_Rodziny(EXTREME,3), #ZROBIC KLASE
    Analiza_Rodziny(GREY,4),
    Analiza_Rodziny(HUDSON,3,2),
    Analiza_Rodziny(HORIZON_PV,1,1),
    # Analiza_Rodziny(KELLY), #POPRAWIC KLASE!!!!
    Analiza_Rodziny(LENOX,4),
    Analiza_Rodziny(MAXWELL_P,2,2),
    # Analiza_Rodziny(MYSTIC,3), #NIE/ZROBIC KLASE?
    Analiza_Rodziny(ONYX,3,2),
    Analiza_Rodziny(OVAL_P,4),
    Analiza_Rodziny(OXYGEN_P,2),
    Analiza_Rodziny(RITZ,3),
    Analiza_Rodziny(WILLOW,2,2),
    Analiza_Rodziny(REVERSO,2,1),
    Analiza_Rodziny(SPECTRA,3),
    Analiza_Rodziny(SAMOA,3,2),
    Analiza_Rodziny(UNO,4),
    Analiza_Rodziny(STONE,1,1)]

ar_podsum = pd.DataFrame([x.Raport() for x in ar])

In [ ]:
ard = {a.MODEL: a for a in ar}

In [ ]:
ar_podsum.sort_values(by=["GRUPA", "OBJ_BRYL_DO_ZAM_DO_OBJ_MAX"], ascending=[True,False])

In [ ]:
podsumowanie_VOL = ar_podsum[["OBJ_CIECH",	"OBJ_VITA",	"OBJ_PIANPOL"]].sum()
podsumowanie_VOL["RAZEM"] = podsumowanie_VOL.sum()
podsumowanie_VOL

In [ ]:
ard["ELIXIR"].Raport("prtW")

#BRYŁY DO ZAMÓWIENIA

In [ ]:
# @title BRYŁY DO ZAMÓWIENIA FUNKCJE
def oblicz_owaty_do_zamowienia(*z_bdz):

  df_zam = pd.concat(z_bdz)
  df_zam = df_zam.merge(owaty, how="left", on="OPIS")

  for c in owaty.columns[1:]:
    df_zam[c] = df_zam[c] * df_zam.DO_ZAMOWIENIA * 1.1

  return df_zam#[owaty.columns[1:]]


def zapotrzebowanie_na_owaty(zam_owaty, wyjatki:list):
  _wyjatki = pd.Series({"O3":0, "O2":0, "O1":0, "L1":0, "W3":0})
  for w in wyjatki:
    _wyjatki += w

  zap = zam_owaty[["O3", "O2", "O1", "L1", "W3"]].sum()+_wyjatki

  # zap *= 1.1

  print(f"O1 zielona: {(zap['O1']/50).round(0):.0f} rolek")
  print(f"O2 niebieska: {(zap['O2']/40).round(0):.0f} rolek")
  print(f"O3 czerwona: {(zap['O3']/40).round(0):.0f} rolek")
  # return zap



def pobierz_zamowienie_z_ZAM_PIANKI(tydzien, _cls):
  _zam = zam_pianki[(zam_pianki.TYDZIEN == tydzien) & (zam_pianki.OPIS.str.contains(_cls.MODEL))].merge(komplety_pianek[["KOD", "BRYLA_GEN"]])[["OPIS", "BRYLA_GEN", "ILE_ZAMOWIONE"]]
  return _cls({i[1].BRYLA_GEN: i[1].ILE_ZAMOWIONE for i in _zam[["BRYLA_GEN", "ILE_ZAMOWIONE"]].iterrows()})



##KOMPONOWANIE ZAMOWIENIA

In [ ]:
#dos 24/0242 sto 2_24, hor 3_24, nr_sam VITA 7_24,  nr partii 08/01
zsto,sto = ard["STONE"].Bryly_do_zamowienia(wszystkie_bryly=True, korekta_zam={
    '1,5': 150.0,
 '2,5': 25.0,
 '3': 45.0,
 '4': 10.0,
#  '5': 0.0,
 'ON': 15.0,
#  'NW140x150': 5.0,
#  '': 50.0,
 '[LH': 25.0,
 'E30': 10.0,
 'P106x106': 10.0,
 'P106x82': 20.0,
#  'P82x82': 2.0,
#  'Z50': 1.0,
 'Z70': 190.0,
 'STONE ][': 250.0})



zhor, hor = ard["HORIZON"].Bryly_do_zamowienia(wszystkie_bryly=True, korekta_zam={
 'ONz': 20.0,
})

sto, hor# + blok t35 1.2x1.2x2 +hr3020

In [ ]:
ard["WILLOW"].Bryly_do_zamowienia(wszystkie_bryly=True, korekta_zam={
#     '1': 35.0,
#  '1,5': 3.0,
#  '2,5': 1.0,
#  '4': 15.0,
#  'N': 0.0,
#  '[L': 9.0,
#  '[L2Hz': 0.0,
#  'P106x106': 0.0,
#  'P82x106': 6.0,
#  'P82x82': 6.0,
#  '[': 26.0,
 ']': 0.0,
 'OWA 1][': 40.0,
 'OWA 1': 20.0,
 'OWA 1,5][': 70.0,
 'OWA 1,5': 50.0,
 'OWA 2,5][': 10.0,
 'OWA 2,5': 10.0,
 'OWA 4][': 25.0,
 'OWA 4': 0.0})

In [ ]:
sto+hor

In [ ]:
1.2*1.2*2*3


In [ ]:
ozn_owat

In [ ]:
zapotrzebowanie_na_owaty(oblicz_owaty_do_zamowienia(zsto, zhor),[])

pd.concat([zsto, zhor]).merge(
    oblicz_owaty_do_zamowienia(zsto, zhor)[["KOD", "O1", "O2", "O3", "L1","W3"]].fillna(0),
    how="left",
    on="KOD").rename(columns={"O1":"ZIELONA", "O2":"NIEBIESKA", "O3":"CZERWONA", "L1":"ŻÓŁTA"})#.to_excel("PIANPOL_2408.xlsx")

In [ ]:
ozn_owat

##DODANIE ZAMÓWIENIA DO BAZY

In [ ]:
pozycje = list()


def Dodaj_pozycje_do_ZAM_PIANKI(tydzien, zancznik_dostawcy, nr_kompletacji, modele:pd.DataFrame, klasa, zam1=None, zam2=None, nr_partii=None, DODAJ_DO_BAZY = False):

  model = klasa.MODEL
  galanteria = klasa.galanteria
  siedziska_HR = klasa.siedziska_HR
  leniwa = klasa.leniwa

  for r in modele.iterrows():
    # tydzien, model, kod, opis, ile_zam, znacznik_dostawcy, galanteria, siedziska_HR, leniwa, nr_kompletacji=None, zam1=None, zam2=None, nr_partii=None
    pozycje.append([tydzien, model, r[1].KOD, r[1].OPIS, r[1].DO_ZAMOWIENIA, zancznik_dostawcy, galanteria, siedziska_HR, leniwa, nr_kompletacji, zam1, zam2, "nr_partii: " + nr_partii])

  if DODAJ_DO_BAZY:

    for r in modele.iterrows():
      session.add(ZAM_PIANKI(tydzien, model, r[1].KOD, r[1].OPIS, r[1].DO_ZAMOWIENIA, zancznik_dostawcy, galanteria, siedziska_HR, leniwa, nr_kompletacji, zam1, zam2, "nr_partii: " + nr_partii))

    session.commit()


#dos 24/0242 sto 2_24, hor 3_24, nr_sam VITA 7_24,  nr partii 08/01

# Dodaj_pozycje_do_ZAM_PIANKI(2407, "P", "2_24", zoxy, oxy, "24/0216", nr_partii="07/01,")#, DODAJ_DO_BAZY=True)

pd.DataFrame(pozycje)

In [ ]:
with engine.begin() as conn:
  zam_pianki_db = pd.read_sql(text("SELECT * from ZAM_PIANKI ORDER BY TYDZIEN DESC LIMIT 30"), conn) # ORDER BY TYDZIEN DESC LIMIT 30

zam_pianki_db[["LP", "TYDZIEN", "OPIS", "ILE_ZAMOWIONE", "ZNACZNIK_DOSTAWCY", "GALANTERIA", "SIEDZISKA_HR", "LENIWA", "ZAM1", "ZAM2", "UWAGI"]]

#RAPORTY DO ZAMÓWIENIA

In [ ]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side
from datetime import datetime as dt

In [ ]:
_tab = tab.copy()
_tab["vol"] = _tab.DLUG*_tab.SZER*_tab.WYS/1000_000_000
_tab["VOL"] = _tab.vol*_tab.ilosc

In [ ]:
dostawca = "CIECH"
with engine.begin() as conn:
  wnd = pd.read_sql(text("SELECT KOD, OPIS, MODEL, NR_KOMPLETACJI, ILE_ZAMOWIONE, ZNACZNIK_DOSTAWCY, ZAM1, ZAM2, GALANTERIA, SIEDZISKA_HR, LENIWA, UWAGI, nr_SAMOCHODU from ZAM_PIANKI WHERE ZAM1 = '24/0151'"), conn)

wnd = wnd.merge(komplety_pianek[["KOD", "BRYLA_GEN", "obj"]], how="left", on="KOD")



def obj_typ(m,bg,i,g,s,l):

    war_len = (_tab.TYP == "G-401")
    war_shr = (_tab.TYP.str.contains("HR|EE"))

    gal = _tab[(_tab.MODEL == m) & (_tab.BRYLA == bg) & ~war_len & ~war_shr].VOL.sum()*i if g == dostawca[0] else 0
    shr = _tab[(_tab.MODEL == m) & (_tab.BRYLA == bg) & (war_shr)].VOL.sum()*i if s == dostawca[0] else 0
    len = _tab[(_tab.MODEL == m) & (_tab.BRYLA == bg) & (war_len)].VOL.sum()*i if l == dostawca[0] else 0

    # return f"{gal} + {shr} + {len}"
    return gal + shr + len


wnd["OBJ_DOSTAWA"] = wnd.apply(lambda x: obj_typ(x.MODEL, x.BRYLA_GEN, x.ILE_ZAMOWIONE, x.GALANTERIA, x.SIEDZISKA_HR, x.LENIWA), axis=1)
wnd["ILE_WOZKOW"] = (wnd.OBJ_DOSTAWA / 5.5)
wnd["ILE_WOZKOW"] = wnd["ILE_WOZKOW"].apply(np.ceil)
wnd["OBJ_KOMPLETACJA"] = wnd.ILE_ZAMOWIONE * wnd.obj
wnd[["MODEL", "NR_KOMPLETACJI", "OPIS", "ILE_ZAMOWIONE", "OBJ_DOSTAWA", "ILE_WOZKOW"]]#.to_excel("ILE_PRZYJECHALO_24_0151.xlsx")
# wnd

In [ ]:
zam_pianki

In [ ]:
from analiza_pianek import _owaty

def zlecenia_produkcyjne_pianki_owaty(model, nr_kompletacji, nr_partii):
  """
  nr_partii -> numer tygodnia / np.: 07/1
  """
  zp = zam_pianki[(zam_pianki.OPIS.str.contains(model))&(zam_pianki.NR_KOMPLETACJI == nr_kompletacji)][["OPIS", "ILE_ZAMOWIONE"]]

  for p in range(zp.shape[0]):

    ow = _owaty[_owaty.OPIS == zp.iloc[p].OPIS][["TYP_OWATY", "ZUZYCIE", "RODZAJ_CIECIA", "NAZWA_UKL"]].reset_index()
    ow["ZUZYCIE"] = ow.ZUZYCIE*zp.iloc[p].ILE_ZAMOWIONE*1.1
    ow["KATER_UKL"] = ow.apply(lambda x: x.NAZWA_UKL if x.RODZAJ_CIECIA == "K" else "", axis=1)

    wb = openpyxl.Workbook()
    sheet = wb.active

    sheet["A1"] = "ZLECENIE PRODUKCYJNE: WYDZIAŁ ROZKRÓJ OWAT / KOMPLETACJA OWATY"
    sheet["A3"] = f"NR PARTII: {nr_partii}"
    sheet["A4"] = f"NR ZAMÓWIENIA: {model} {nr_kompletacji}"
    sheet["A5"] = f"MODEL: {model}"
    sheet["H1"] = f"DZIEŃ WYDRUKU      {dt.now().strftime('%Y-%m-%d')}"
    sheet["E4"] = "TYP"
    sheet["F4"] = "MB"
    sheet["G4"] = "R/K"
    sheet["H4"] = "KATER_UKL"

    start_row = 5
    for r in ow.iterrows():

      sheet.cell(row=r[0]+start_row, column=5, value=r[1].TYP_OWATY)
      sheet.cell(row=r[0]+start_row, column=6, value=r[1].ZUZYCIE)
      sheet.cell(row=r[0]+start_row, column=7, value=r[1].RODZAJ_CIECIA)
      sheet.cell(row=r[0]+start_row, column=8, value=r[1].KATER_UKL)


    sheet["A15"] = "BRYŁA"
    sheet["B15"] = "ILOŚĆ PACZEK"
    sheet["C15"] = "CZAS"
    sheet["A16"] = zp.iloc[p].OPIS
    sheet["B16"] = zp.iloc[p].ILE_ZAMOWIONE

    sheet["A18"] = "UWAGI:"
    sheet["H18"] = "PODPIS, DZIEN:"

    thin = Side(border_style="thin", color="000000")
    for row in sheet[f"E4:i{ow.shape[0]+4}"]:
        for cell in row:
          cell.border = Border(bottom=thin)

    wb.save(f"OWATY/OWATY {model} {nr_kompletacji} {zp.iloc[p].OPIS.replace(model, '')}.xlsx")


def zlecenia_produkcyjne_pianki_kompletacja(model, nr_kompletacji, nr_partii):
  """
  nr_partii -> numer tygodnia / np.: 07/1
  """
  zp = zam_pianki[(zam_pianki.OPIS.str.contains(model))&(zam_pianki.NR_KOMPLETACJI == nr_kompletacji)][["OPIS", "ILE_ZAMOWIONE"]]

  for p in range(zp.shape[0]):

    ow = _owaty[_owaty.OPIS == zp.iloc[p].OPIS][["TYP_OWATY", "ZUZYCIE", "RODZAJ_CIECIA", "NAZWA_UKL"]].reset_index()
    ow["ZUZYCIE"] = ow.ZUZYCIE*zp.iloc[p].ILE_ZAMOWIONE*1.1
    ow["KATER_UKL"] = ow.apply(lambda x: x.NAZWA_UKL if x.RODZAJ_CIECIA == "K" else "", axis=1)

    wb = openpyxl.Workbook()
    sheet = wb.active

    sheet["A1"] = "ZLECENIE PRODUKCYJNE: KOMPLETACJA PIANKI"
    sheet["A3"] = f"NR PARTII: {nr_partii}"
    sheet["A4"] = f"NR ZAMÓWIENIA: {model} {nr_kompletacji}"
    sheet["A5"] = f"MODEL: {model}"
    sheet["H1"] = f"DZIEŃ WYDRUKU      {dt.now().strftime('%Y-%m-%d')}"

    sheet["A15"] = "BRYŁA"
    sheet["B15"] = "ILOŚĆ PACZEK"
    sheet["C15"] = "CZAS"
    sheet["A16"] = zp.iloc[p].OPIS
    sheet["B16"] = zp.iloc[p].ILE_ZAMOWIONE

    sheet["A18"] = "UWAGI:"
    sheet["H18"] = "PODPIS, DZIEN:"



    wb.save(f"KOMPLETACJA/KOMPLETACJA {model} {nr_kompletacji} {zp.iloc[p].OPIS.replace(model, '')}.xlsx")



In [ ]:
# zlecenia_produkcyjne_pianki_owaty("WIL", "owa_1", "")


In [ ]:
# @title RAPORTY DO ZAMÓWIENIA FUNKCJE

def specyfikacja_zam_vita_xlsx(NR_zam:str, raport_vita:pd.DataFrame):
  wb = openpyxl.Workbook()
  sheet = wb.active

  sheet["B1"] = f"SPECYFIKACJA ZAMÓWIENIE DOS/{NR_zam}"
  sheet["H1"] = dt.now().strftime("%Y-%m-%d")
  sheet["B3"] = "Zamawiajacy"
  sheet["B4"] = "OLTA K.K. Zawistowscy sp. j"
  sheet["B5"] = "Ignatki 40/6"
  sheet["B6"] = "16-001 Kleosin"
  sheet["B7"] = "NIP 966 14 08 783"
  sheet["H3"] = "Dostawca"
  sheet["H4"] = "VITA POLYMERS POLAND SP. Z O.O."
  sheet["A8"] = "LP"
  sheet["I8"] = "UWAGI"

  rows = dataframe_to_rows(raport_vita)
  for r_idx, row in enumerate(rows, 1):
      for c_idx, value in enumerate(row, 1):
          sheet.cell(row=r_idx+7, column=c_idx, value=value)

  thin = Side(border_style="thin", color="000000")
  for row in sheet[f"A9:i{raport_vita.shape[0]+9}"]:
      for cell in row:
        cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

  wb.save(f"SPECYFIKACJA DO ZAMÓWIENIA DOS {' '.join(NR_zam.split('/'))}.xlsx")

def raport_vita(*args):
  """
  Zesatwaienie zamówionych pianek do VITA ilośiowe i z podziałem na bryły
  """


  pianki = [x.vita for x in args]
  zam_pianek_vita = pd.concat(pianki).fillna("")[["TYP", "NUMER", "ilosc", "PROFIL", "OZN", "OPIS", "WYMIAR"]]
  zam_pianek_vita.set_index(pd.Index([x for x in range(1,zam_pianek_vita.shape[0]+1)]),inplace=True)

  podsumowanie_zamowienia_vita = pd.concat(pianki).fillna("").sort_values(by="NUMER")
  podsumowanie_zamowienia_vita.set_index(pd.Index([x for x in range(1,podsumowanie_zamowienia_vita.shape[0]+1)]),inplace=True)



  return zam_pianek_vita, podsumowanie_zamowienia_vita[["TYP", "NUMER", "ilosc", "VOL", "PROFIL", "OZN", "OPIS", "WYMIAR"] + [x for x in podsumowanie_zamowienia_vita.columns if "br" in x]]


def raport_dostarczonych_pianek(cls, drukuj_excel=False):
  """
  Zesatwianie ilosci pianek do modelu z dostawy
  komplety_pianek -> df z kopletami pianek
  cls -> zinicjalizowana instacja klasy zawierająca odpowiedni model
  """
  zestawienie_pianek_do_bryly = list()

  for br in cls.bryly.keys():
    df = tab[(tab.MODEL == cls.MODEL) & (tab.BRYLA == br)]
    df["ILOŚĆ"] = (df.ilosc * cls.bryly[br]).astype(int)
    df["DOSTARCZONO/UWAGI"] = ""
    df = df[['TYP', 'PRZEZ', 'OR', 'OZN', 'PROFIL', 'NUMER', 'WYMIAR', 'ILOŚĆ', 'DOSTARCZONO/UWAGI']].fillna("-")
    df.set_index(pd.Index([x for x in range(1,df.shape[0]+1)]),inplace=True)
    zestawienie_pianek_do_bryly.append([br, df])

  def drukuj_zestawienie_dla_bryly_xlsx(zpdb_n):
    _df = zpdb_n[1]
    # _df = _df[_df.TYP != "G-401"]
    header = f"{cls.MODEL} {zpdb_n[0]} - {cls.bryly[zpdb_n[0]]:.0f}szt"
    print(header)
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.oddHeader.left.text = header
    sheet.oddHeader.left.size = 18
    sheet.oddHeader.left.font = "Calibry,Bold"
    # sheet.oddHeader.right.text = "Data dostawy:            \nNr dostawy:           "
    sheet.append(["LP"]+list(_df.columns))
    rows = dataframe_to_rows(_df,header=False)
    for r in list(rows)[1:]:
      sheet.append(r)

    thin = Side(border_style="thin", color="000000")
    sheet.page_setup.orientation = sheet.ORIENTATION_LANDSCAPE
    for row in sheet[f"A1:J{_df.shape[0]+1}"]:
      for cell in row:
        cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

    wb.save(f"{cls.MODEL[:3]} {zpdb_n[0]}.xlsx")

  if drukuj_excel:
    for i in zestawienie_pianek_do_bryly:
      drukuj_zestawienie_dla_bryly_xlsx(i)
    return 0

  return zestawienie_pianek_do_bryly

In [ ]:
r = raport_vita(sto, hor)

In [ ]:
r[1]

In [ ]:
specyfikacja_zam_vita_xlsx("24/0242",r[0])

In [ ]:
r[1].to_excel("ZESTAWIENIE PIANEK DO ZAMOWIENIA DOS 24_0242.xlsx")

In [ ]:
# raport_dostarczonych_pianek(sto, drukuj_excel=True)

In [ ]:
# raport_dostarczonych_pianek(hor, drukuj_excel=True)